# Worksheet 08

Name: Ryan Chung  
UID: U33101958

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[3.780352009244771, 6.082787547466665, 3.6097353426749925, 2.8369058567769843, 5.886912719010031, 5.1764691359030435, 5.356688282979044, 5.333967404363398, 6.313572587269787, 3.6979644523103214]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:
s2 = np.random.normal(8,1,10).tolist()
print(s2)

[7.132992656710234, 7.4608679218425795, 6.839446499619022, 8.825112910322757, 8.139104779860283, 6.681269494182282, 8.483999192218986, 8.190893792433492, 7.5645077792401025, 7.913961919138005]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [5]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[7.913961919138005, 3.6979644523103214, 7.5645077792401025, 8.190893792433492, 8.483999192218986, 6.681269494182282, 6.313572587269787, 5.333967404363398, 8.139104779860283, 5.356688282979044]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [8]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2))]
mean = [ sum(s1)/len(s1) ,  sum(s2)/len(s2)]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[3.6979644523103214, 6.681269494182282, 6.313572587269787, 5.333967404363398, 5.356688282979044]
[7.913961919138005, 7.5645077792401025, 8.190893792433492, 8.483999192218986, 8.139104779860283]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 5.4766924442209675,  mean_2 = 8.058493492578174
var_1 = 1.0700037896094234,  var_2 = 0.09399887603381765


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [9]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j), calculate pdf and first item is pdf of first cluster, 2nd item is pdf for 2nd cluster ... 
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]


    # pdf_i[0] is p(p | s_0)
        
    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  7.913961919138005
probability of observing that point if it came from cluster 0 =  0.027852884624638623
probability of observing that point if it came from cluster 1 =  1.30140572477511
point =  3.6979644523103214
probability of observing that point if it came from cluster 0 =  0.09363831242498322
probability of observing that point if it came from cluster 1 =  0.0
point =  7.5645077792401025
probability of observing that point if it came from cluster 0 =  0.05556314814542237
probability of observing that point if it came from cluster 1 =  4.27303052362282e-06
point =  8.190893792433492
probability of observing that point if it came from cluster 0 =  0.014938089344218742
probability of observing that point if it came from cluster 1 =  1.5738968218003908
point =  8.483999192218986
probability of observing that point if it came from cluster 0 =  0.007181720291060641
probability of observing that point if it came from cluster 1 =  0.00015073004850893657
point =  6.681269494182282

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [10]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x) , sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)  ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 6.20721477354324,  mean_2 = 8.084799614195742
var_1 = 2.146263513036785,  var_2 = 3.5407217139670806


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)